# Generador de instancias
En este notebook está el código para generar los sets de instancias que se usan para experimentar.
- Estas instancias van a ser guardadas en la carpeta __instancias__.
- Cada set estará en su propia carpeta y tendrá un archivo _indice.csv_ que contendrá información sobre las instancias.

In [9]:
import random, math
import pandas as pd
import numpy as np
import os

In [10]:
def save_instance(dataset, instance_name, n, mG, mH, interseccion, edges, i):
    instance_name = f"{instance_name}-{n}-{mG}-{mH}-{interseccion}-{i}";
    directory = f"..\\instances\\{dataset}"
    if not os.path.exists(directory):
        os.makedirs(directory)
    with open(f"{directory}\\{instance_name}.in", "w+") as f:
        print(n, mG, mH, file=f)
        for edge in edges: 
            print(f"{edge[0]} {edge[1]}\n", file=f, end="")

In [11]:
from pathlib import Path

def save_row(filas_indice, dataset, instance_name, n, mG, mH, interseccion, i):
    filas_indice.append([dataset, instance_name+f"-{n}-{mG}-{mH}-{interseccion}-{i}", n, mG, mH, interseccion, i, "..\\instances\\"+ dataset + "\\"+ instance_name+f"-{n}-{mG}-{mH}-{interseccion}-{i}.in"])

def save_to_csv(filas_indice,dataset):
    df = pd.DataFrame(filas_indice, columns=["dataset", "instancia", "n", "mG", "mH","interseccion","i", "archivo"])
    Path("..\\instances\\"+dataset).mkdir(parents=True, exist_ok=True)
    df.to_csv(f"..\\instances\\{dataset}\\indice.csv", index=False, header=True)

In [12]:
random.seed(42)

## Optimo

In [13]:
from os import listdir
from os.path import isfile, join

dataset = "optimo"
instance_name = "optimo"
filas_indice = []

path = "../../instancias"; 
onlyfiles = [join(path,f) for f in listdir(path) if isfile(join(path, f)) and f.find(".in") != -1]

for file in onlyfiles:
    with open(file) as f:
        n, mG, mH = [int(x) for x in next(f).split()] # read first line
        edges = []
        for line in f: # read rest of lines
            edges.append([int(x) for x in line.split()])
            
        save_instance(dataset,instance_name, n, mG, mH, 0, edges, 1)
        save_row(filas_indice, dataset, instance_name, n, mG, mH, 0, 1)
        
save_to_csv(filas_indice,dataset)

## Generar instancias Random Densas.

Guardar instancias que tienen distintas intersecciones en la misma carpeta

In [14]:
def generarFamilia(dataset,instance_name,nList,fmG,fmH,interseccion,cantReps):
    filas_indice = []
    
    for n in nList:
        mG = fmG(n)
        mH = fmH(n)
        listaAristas = [(nodo1,nodo2) for nodo1 in range(1 , n+1) for nodo2 in range(1, n+1) if nodo1<nodo2]
    
        for i in range(cantReps): 
     
            #Armo G y H densos para n nodos.
            random.shuffle(listaAristas)
            
            minimoM = min(mG,mH)
            cantidadAristas = int(minimoM * (interseccion / 100))
            
            aristasInterseccion = listaAristas[:cantidadAristas]
            aristasG = listaAristas[cantidadAristas:mG] + aristasInterseccion
            aristasH = listaAristas[mG:mG+mH-cantidadAristas] + aristasInterseccion
            
            save_instance(dataset,instance_name, n, mG, mH, interseccion, aristasG + aristasH, i)
            save_row(filas_indice, dataset, instance_name, n, mG, mH, interseccion, i)
    save_to_csv(filas_indice,dataset)


Guardar instancias que tienen distintas intersecciones en distintas carpetas

In [15]:
def generarFamiliaInterseccion(dataset,instance_name,nList,fmG,fmH,interseccion,cantReps):
    filas_indice = []
    
    for n in nList:
        mG = fmG(n)
        mH = fmH(n)
        listaAristas = [(nodo1,nodo2) for nodo1 in range(1 , n+1) for nodo2 in range(1, n+1) if nodo1<nodo2]
    
        for i in range(cantReps): 
     
            #Armo G y H densos para n nodos.
            random.shuffle(listaAristas)
            
            minimoM = min(mG,mH)
            cantidadAristas = int(minimoM * (interseccion / 100))
            
            aristasInterseccion = listaAristas[:cantidadAristas]
            aristasG = listaAristas[cantidadAristas:mG] + aristasInterseccion
            aristasH = listaAristas[mG:mG+mH-cantidadAristas] + aristasInterseccion
            
            save_instance(f"{dataset}-{interseccion}",instance_name, n, mG, mH, interseccion, aristasG + aristasH, i)
            save_row(filas_indice, f"{dataset}-{interseccion}", instance_name, n, mG, mH, interseccion, i)
    save_to_csv(filas_indice,f"{dataset}-{interseccion}")
    

In [16]:
from os import listdir
from os.path import isfile, join


dataset_denso = "random-denso-denso"
instance_name_denso = "random-densos-denso"

dataset_ralo = "random-ralo-ralo"
instance_name_ralo = "random-ralo-ralo"

dataset_ralo_denso = "random-ralo-denso"
instance_name_ralo_denso = "random-ralo-denso"

dataset_denso_ralo = "random-denso-ralo"
instance_name_denso_ralo = "random-denso-ralo"

f_denso = lambda n : int(n*(n-1)/4)
f_ralo = lambda n : int(2*n)
   
datasets = [
    ("random-denso-denso",f_denso,f_denso),
    ("random-ralo-ralo",f_ralo,f_ralo),
    ("random-ralo-denso",f_ralo,f_denso),
    ("random-denso-ralo",f_denso,f_ralo)
]

cantidadReps = 5
minNumeroNodos = 10
maxNumeroNodos = 85

nList = [n for n in range(minNumeroNodos,maxNumeroNodos,10)]
intersecciones = [25,75]

for dataset, fmG, fmH in datasets:
    for interseccion in intersecciones:
        generarFamilia(dataset,dataset,nList,fmG,fmH,interseccion,cantidadReps)

In [17]:
from os import listdir
from os.path import isfile, join


dataset_denso = "random-denso-denso"
instance_name_denso = "random-densos-denso"

dataset_ralo = "random-ralo-ralo"
instance_name_ralo = "random-ralo-ralo"

dataset_ralo_denso = "random-ralo-denso"
instance_name_ralo_denso = "random-ralo-denso"

dataset_denso_ralo = "random-denso-ralo"
instance_name_denso_ralo = "random-denso-ralo"

f_denso = lambda n : int(n*(n-1)/4)
f_ralo = lambda n : int(2*n)
   
datasets = [
    ("random-denso-denso",f_denso,f_denso),
    ("random-ralo-ralo",f_ralo,f_ralo),
    ("random-ralo-denso",f_ralo,f_denso),
    ("random-denso-ralo",f_denso,f_ralo)
]

cantidadReps = 3
minNumeroNodos = 10
maxNumeroNodos = 101

nList = [n for n in range(minNumeroNodos,maxNumeroNodos,10)]
intersecciones = [25,75]

for dataset, fmG, fmH in datasets:
    for interseccion in intersecciones:
        generarFamiliaInterseccion(dataset,dataset,nList,fmG,fmH,interseccion,cantidadReps)